In [401]:
from pymongo import MongoClient
from pprint import pprint
import re

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
from pprint import pprint
import itertools

# MongoDB

In [403]:
client = MongoClient('localhost',27017)

In [404]:
#указатель на базу данных 
db = client['Jobs_and_salary']

In [405]:
jobs_hh = db.hh

# Подключаемся к сайту hh.ru

In [406]:
def max_number_page_HH():
    
    
    """"
    Возвращает максимальное количество страниц на сайте. 
    если страница одна , то блока со страницами нет, тогда возвраащет [1] 
    
    """
    
    main_link_hh = 'https://hh.ru/'

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                            AppleWebKit/537.36 (KHTML, like Gecko)\
                            Chrome/83.0.4103.116 Safari/537.36'}



    params = {'order_by':'publication_time',
         'items_on_page':100,
          "currency_code":"RUR",
          "area":113
         }

    html = requests.get(main_link_hh + 'search/vacancy/' , headers = headers, params = params)

    soup = bs(html.text,'html.parser')

    count_block = soup.find('div',{'data-qa':'pager-block'})
    
    if count_block:
        
        max_count_list = count_block.find_all('a',{'class':'bloko-button'})[-2].getText()
        
    else:
        
        return np.array(1)
    

    return np.arange(1,int(max_count_list)+1)

In [407]:
max_number_page_HH()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [408]:
all_pages = max_number_page_HH()

In [409]:


for page in all_pages:
    params = {'order_by':'publication_time',
         'items_on_page':100,
          "currency_code":"RUR",
          "area":113,
        'page':page-1
         }

        
    main_link_hh = 'https://hh.ru'

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                                AppleWebKit/537.36 (KHTML, like Gecko)\
                                Chrome/83.0.4103.116 Safari/537.36'}
    
    response = requests.get(main_link_hh +'/search/vacancy', headers = headers, params = params)


    soup = bs(response.text,'html.parser')

    vacancy_block = soup.find('div',{'class':'vacancy-serp'})

    vacancy_pars = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})


    for i in vacancy_pars:
        ##
        vacancy_name = i.find('a',{'class':'bloko-link'}).getText()
        vacancy_name = vacancy_name.replace('\xa0','')

        ##
        vacancy_id = i.find('a', {'class':'search-resume-item-label'})['href'].split('=')[-1]


        ##
        vacancy_employer = i.find('div',{'class':'vacancy-serp-item__meta-info'}).getText()


        ##
        employer_id = i.find('a',{'class':'bloko-link'})['href']

        if '?' in employer_id:
            employer_id = employer_id[:employer_id.index('?')].split('/')[-1]

        else:
            employer_id = employer_id.split('/')[-1]


        ##
        city = i.find('span',{'class':'vacancy-serp-item__meta-info'}).getText()
        city = city.split(', ')[0]


        ##
        salary_all = i.find('div',{'class':'vacancy-serp-item__sidebar'})
        min_salary = np.nan
        max_salary = np.nan
        
        if salary_all.getText():
            salary_all = salary_all.getText()
            currency = salary_all[-4:]
            salary_all = salary_all[:-4]
            if '-' in salary_all:
                min_salary = salary_all.split('-')[0].replace('\xa0','')
                max_salary = salary_all.split('-')[1].replace('\xa0','')
            elif 'от' in salary_all:
                min_salary = salary_all.replace('от ','')
                min_salary = min_salary.replace('\xa0','')
                
            elif 'до' in salary_all:
                max_salary = salary_all.replace('до ','')
                max_salary = max_salary.replace('\xa0','')
                max_salary = max_salary.replace(' ','')
        else:
            salary_all = np.nan
            currency = np.nan  
            
            
        # Задание 3
        # Использовал try для исключение одинаковых id по вакансиям. 
        # есдинсвенную ошибку могу поймать если _id уже существует
        # И при новом запуске я добавляю вакансии которых еще нет у меня остальные не добалсяются 
        try:
            jobs_hh.insert_one({'_id':vacancy_id,
                                'vacancion_name':vacancy_name,
                                'min_salary':float(min_salary),
                                'max_salary': float(max_salary),
                                'city':city,
                                'company':vacancy_employer,
                                'currency':currency,
                                'website':'hh.ru'})
        except:
            continue
print('OK')

OK


In [410]:
# Счетчик для вакансий
# показано , сколько всего вакансия собрано на момент 06.07.2020 в 20:15
j = 0
for i in jobs_hh.find({}):
    j+=1
print(j)

2040


# Задание 2

In [411]:
my_salary = int(input('Введите ЗП: '))

Введите ЗП: 230000


In [412]:
for i in jobs_hh.find( {'$or': [ { 'max_salary': { '$gt': my_salary } }, { 'min_salary':{'$gt':my_salary}} ] }):
    pprint(i)

{'_id': '36880808',
 'city': 'Москва',
 'company': 'ООО Московский Аукционный Дом Недвижимости\xa0',
 'currency': 'руб.',
 'max_salary': 700000.0,
 'min_salary': 200000.0,
 'vacancion_name': 'Экспертпо продажам премиальной недвижимости',
 'website': 'hh.ru'}
{'_id': '37853687',
 'city': 'Москва',
 'company': 'ООО Национальная Академия Современных Технологий г. Москва\xa0',
 'currency': 'руб.',
 'max_salary': 450000.0,
 'min_salary': 70000.0,
 'vacancion_name': 'Менеджер отдела продаж',
 'website': 'hh.ru'}
{'_id': '37852822',
 'city': 'Москва',
 'company': 'ООО Ликвид\xa0',
 'currency': 'руб.',
 'max_salary': 300000.0,
 'min_salary': 80000.0,
 'vacancion_name': 'Агент по недвижимости',
 'website': 'hh.ru'}
{'_id': '37661227',
 'city': 'Воронеж',
 'company': 'ООО Дом Для Вас\xa0',
 'currency': 'руб.',
 'max_salary': 400000.0,
 'min_salary': 50000.0,
 'vacancion_name': 'Строитель-универсал',
 'website': 'hh.ru'}
{'_id': '36649951',
 'city': 'Москва',
 'company': 'ООО ФИНЭКСПРЕСС\xa0',
 '

### к SuperJob возвращаться даже не хочу, все то что было написано раньше, по нему, полностью обнулилось и ошибка на ошибке 